# Setup


In [ ]:
import weaviate

client = weaviate.connect_to_local()

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="title_vector",
            # source_properties=[""],
            api_endpoint="http://ollama:11434", # If using Docker, use this to contact your local Ollama instance
            model="snowflake-arctic-embed",     # The model to use, e.g. "nomic-embed-text"
        )
    ],

    generative_config=Configure.Generative.ollama(
        api_endpoint="http://ollama:11434",  # If using Docker, use this to contact your local Ollama instance
        model="llama3"                       # The model to use, e.g. "phi3", or "mistral", "command-r-plus", "gemma"
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [ ]:
papers = client.collections.get("Papers")
papers.tenants.get()